# Fake Injection

Author: Shu Liu
Last Edited Date: 05/05/2020

## Abstract
This tutorial illustrates how to inject fake psf to a `calexp` image. The injection method is from: https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/insertFakes.py#L456

In [1]:
from lsst.daf.persistence import Butler
import lsst.geom as geom
import lsst.afw.image as afwImage

In [2]:
# Setup data id
tract, patch, filt = 4639, '0,0', 'r'
visit, detector = 181868, 57
calexp_id = {'visit': visit, 'detector': detector, 'filter': filt}

# Remote calexp information
calexp_repo = '/global/u1/s/shl159/GitHub/fake_data'

# calexp butler, calexp photoCalib
calexp_butler = Butler(calexp_repo)
calexp_photoCalib = calexp_butler.get('calexp_photoCalib',  calexp_id)

In [3]:
# Get calexp exposure from remote
calexp_exposure = calexp_butler.get('calexp', dataId=calexp_id)

In [4]:
# This is the function we use to inject fake psfs to an exposure.

def inject_stars(exposure, photoCalib, x, y, magVar):
    exposure.mask.addMaskPlane("FAKE")
    bitmask = exposure.mask.getPlaneBitMask("FAKE")

    xy = geom.Point2D(x, y)
    psf = exposure.getPsf()

    starIm = psf.computeImage(xy)
    # This is the default value in 'mkFakeStars'
    # https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/insertFakes.py#L458
    calibFluxRadius = 12
    correctedFlux = psf.computeApertureFlux(calibFluxRadius, xy)
    starIm /= correctedFlux
    flux = photoCalib.magnitudeToInstFlux(magVar, xy)
    starIm *= flux
    
    fakeImage = starIm.convertF()
    
    imageBBox = exposure.getBBox()
    imageMI = exposure.maskedImage
    
    interpFakeImage = fakeImage
    interpFakeImBBox = fakeImage.getBBox()
    interpFakeImBBox.clip(imageBBox)
    imageMIView = imageMI.Factory(imageMI, interpFakeImBBox)
    
    if interpFakeImBBox.getArea() > 0:
        clippedFakeImage = interpFakeImage.Factory(interpFakeImage, interpFakeImBBox)
        clippedFakeImageMI = afwImage.MaskedImageF(clippedFakeImage)
        clippedFakeImageMI.mask.set(bitmask)
        imageMIView += clippedFakeImageMI


In [5]:
magVar = 20
for x in range(1000, 3000, 400):
    for y in range(1000, 3000, 400):
        inject_stars(calexp_exposure, calexp_photoCalib, x, y, magVar)


In [6]:
# Here we put the fake calexp to the calexp_repo
calexp_butler.put(calexp_exposure, 'calexp', dataId=calexp_id)

In [ ]:
# We can also directly save the image to disc for quick check.
# calexp_exposure.writeFits('calexp_mag20.fits')